# Prepare enviroment

We will use the Scikit-learn library as it will provide us with an interface to apply machine learning techniques.

In [1]:
# we need to use an old version of scikit, otherwise we will face a compatiblity issue with eli5 (needed in order to use permutation importance
# to answer research question #3
# If we do not use this verion, eli5 will try to import a function (if_delegate_has_method) from scikit-learn that no longer exists
# in recent versions of scikit-learn and we will run into an error
!pip install scikit-learn==0.22

In [2]:
!pip install eli5

In [4]:
# model
from sklearn.svm import SVR # this will make a support vector machine for regression

# permutation
import eli5
from eli5.sklearn import PermutationImportance # the downside of SVM is that it is a 'black-box', this will allow us to access our model and compute feauture importance to answer research question #3 without having to resort to do more work to prepare a i.e., decision tree or bayesian network

# scoring metric
from sklearn.metrics import r2_score, mean_squared_error # to evaluate our SVM

# formatting data
from sklearn.preprocessing import scale # scale and center data
from sklearn.preprocessing import StandardScaler

# model selection
from sklearn.model_selection import train_test_split # split data into training and testing sets
from sklearn.model_selection import GridSearchCV # this will do cross validation and allow us to find optimal parameters for our SVM
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold # provides K-Fold Cross Validation, in our case we will have 10 folds (9 subsamples will be used to train the model and the remaining 1 subsample is used to validate the model)

from sklearn.pipeline import make_pipeline

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

ImportError: numpy.core.multiarray failed to import

In [10]:
from IPython.display import display, Image

# data wrangling
import pandas as pd
import numpy as np
from scipy.stats import pearsonr


# visualizations
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import seaborn as sns
from seaborn import histplot

ImportError: this version of pandas is incompatible with numpy < 1.20.3
your numpy version is 1.19.5.
Please upgrade numpy to >= 1.20.3 to use this pandas version

In [12]:
import numpy as np

In [8]:
import platform

In [14]:
import scipy

In [15]:
print('Python version:', platform.python_version())
print('numpy version:', np.__version__)
print('scipy version:', scipy.__version__)

Python version: 3.8.16
numpy version: 1.19.5
scipy version: 1.10.0


# Data Pre-processing

## Import data

In [ ]:
df_RAW_pre2023 = pd.read_csv('pre-2023.csv')
df_RAW_2023 = pd.read_csv('2023.csv')

## Cleaning

### Removing columns we dont care about

Let's remove the columns with variables that are not present in both datasets.

In [ ]:
print(df_RAW_pre2023.columns)
print(df_RAW_2023.columns)

In [ ]:
remove_pre2023 = ['Positive affect', 'Negative affect']

df_RAW_pre2023 = df_RAW_pre2023.drop(columns=remove_pre2023)

remove_2023 = ['Standard error of ladder score', 'upperwhisker', 'lowerwhisker',
              'Ladder score in Dystopia', 'Explained by: Log GDP per capita',
              'Explained by: Social support','Explained by: Healthy life expectancy',
              'Explained by: Freedom to make life choices','Explained by: Generosity',
              'Explained by: Perceptions of corruption','Dystopia + residual']

df_RAW_2023 = df_RAW_2023.drop(columns=remove_2023)

Add a `year` column to `df_2023` so that information is not lost if the dataframe is merged with `df_pre2023` in the future,

In [ ]:
df_RAW_2023.insert(loc=1, column='year', value=2023)
df_RAW_2023.head()

In [ ]:
print(df_RAW_pre2023.columns)
print(df_RAW_2023.columns)

Sanity check: Let's make sure all the dataframes have the same number of columns.

In [ ]:
# List of DataFrames
dataframes = [df_RAW_pre2023, df_RAW_2023]

# Count number of columns in each DataFrame
for i, df in enumerate(dataframes):
    num_columns = df.shape[1]
    print("Number of columns in DataFrame", i+1, ":", num_columns)

### Standardize naming scheme for dimensions

Between the two datasets, there is a slight difference in the naming scheme. We the column names to be the same to allow for merging later on.

In [ ]:
df_RAW_pre2023.rename(columns={
    'Country name': 'country',
    'year': 'year',
    'Life Ladder': 'happiness',
    'Log GDP per capita': 'GDP',
    'Social support' : 'socialSupport',
    'Healthy life expectancy at birth': 'lifeExpectancy',
    'Freedom to make life choices': 'freedom',
    'Generosity': 'generosity',
    'Perceptions of corruption' : 'corruption',
}, inplace=True)

df_RAW_2023.rename(columns={
    'Country name': 'country',
    'year': 'year',
    'Ladder score': 'happiness',
    'Logged GDP per capita': 'GDP',
    'Social support' : 'socialSupport',
    'Healthy life expectancy': 'lifeExpectancy',
    'Freedom to make life choices': 'freedom',
    'Generosity': 'generosity',
    'Perceptions of corruption' : 'corruption',
}, inplace=True)


In [ ]:
print(df_RAW_pre2023.columns)
print(df_RAW_2023.columns)

### Merge

In [ ]:
df = pd.concat([df_RAW_pre2023, df_RAW_2023], axis=0)
df

### Dealing with NaN or 0 values

Let's make sure we do not have any NA values, since Scikit-learn's support vector machines do not work with datasets with missing values.




In [ ]:
df.dtypes # check if there are character-based place holders for missing data

In [ ]:
print("Number of null or missing values: ",df.isnull().sum().sum())

According to the [statistics appendix](https://s3.amazonaws.com/happiness-report/2018/Appendix1ofChapter2.pdf), 0 is not a meaningful score for any of the variables. Any cells that have a value of zero are actually cells with missing data.

Let's check to see if any observation has a 0 as a value.

In [ ]:
zero_indices = np.where(df == 0)
num_zeros = zero_indices[0].shape[0]
print(f'Number of cells with a value of zero: {num_zeros}')

While it seems that some countries have missing data, it is not justifiable to discard the whole observation. In our dataset, usually for country that has missing data, it is only empty for a single variable and still has the 6 other dimensions filled. Therefore, discarding rows with missing information is not a good tradeoff.

Even though some cells have a value of `0` or `NaN`, we can use **interpolation** to estimate a resonable value for these cells and replace these cells.

A linear method of interpolation will draw a straight line between the points surrounding the missing value(s) and use this line to estimate a reasonable value. It's worth noting that we will assumes that the change between the values is constant.

Interpolation works only on cells exclusively with `NaN`. As such, we will replace all 0's with NaN's before proceeding.

In [ ]:
df.replace(0, np.nan)
print("Number of null or missing values: ",df.isnull().sum().sum()) # check to see if we successfully replaced 0's

Now that our dataset exclusively has missing data as `NaN`, we can proceed with the interpolation.

In [ ]:
df.interpolate(method='linear', inplace=True)
print("Number of null or missing values: ",df.isnull().sum().sum())

Success! We have no more `NaN` values in out dataset.

How many years does our dataframe account for? which countries are missing from each year?

In [ ]:
# To identify the countries in df
all_countries = df['country'].unique()
print(f"The dataframe covers the years: {all_countries}")

In [ ]:
# To identify the years in df
years = df['year'].unique()
print(f"The dataframe covers the years: {years}")

# To find out which countries are missing for each year:
for year in years:
    countries_in_year = df[df['year'] == year]['country'].unique()
    missing_countries = set(all_countries) - set(countries_in_year)
    print(f"In the year {year}, the following countries are missing: {missing_countries}")


# **Q1:** Can we predict this year's (2023) happiness?

Happiness is considered an indicator of societal health. If a government can predict its citizens' happiness accurately, it can use that information to evaluate the success of its current policies and make adjustments where necessary.

We already have the 2023 data. What we can do is train a model on the pre-2023 data to see if the predicted 2023 scores actually match the actual ones. If the model we build has a high accuracy, we can forecast 2024 scores!

Our ultimate goal is to predict the 2023 data, and were interested in seeing how well our model performs specifically on this year, we wil exclude the 2023 data from our training and testing sets entirely. We will then train our model on the pre-2023 data, and once the model is trained, use it to make predictions on the 2023 data.

## Pre-processing for this specific model

In [ ]:
df_pre2023 = df[df['year'] < 2023] # we will train and test our model on this dataframe
df_2023 = df[df['year'] == 2023] # well compare predicted data with this dataframe

**Why did we choose SVM?**

### Dealing with countries present in pre-2023 data, but not 2023 (or vice versa)


Some countries present in your training data are not present in the 2023 data, which cause an issue in our model later on as it expects to see the same features (in this case, countries) in both the training and testing data.

In [ ]:
# Get a list of all unique countries in each dataframe
countries_pre2023 = set(df_pre2023['country'].unique())
countries_2023 = set(df_2023['country'].unique())

# Find countries present in pre-2023 data but not 2023
missing_in_2023 = countries_pre2023 - countries_2023
print("Countries present in pre-2023 data but not 2023:")
print(missing_in_2023)

# Find countries present in 2023 data but not pre-2023
missing_in_pre2023 = countries_2023 - countries_pre2023
print("\nCountries present in 2023 data but not pre-2023:")
print(missing_in_pre2023)

It turns out the countries the 2023 dataframe is missing are:

 'Azerbaijan', 'Turkmenistan', 'Qatar', 'Angola', 'Belize', 'Oman', 'Belarus', 'Bhutan', 'Libya', 'Kuwait', 'Trinidad and Tobago', 'Djibouti', 'Maldives', 'Cuba', 'Haiti', 'Rwanda', 'South Sudan', 'Syria', 'Central African Republic', 'Eswatini', 'Somaliland region', 'Guyana', 'Yemen', 'Suriname', 'Burundi', 'Lesotho', 'Sudan', 'Somalia'

Hence, we need to remove these countries from the pre-2023 dataframe.

In [ ]:
df_pre2023

In [ ]:
remove_countries = ['Azerbaijan', 'Turkmenistan', 'Qatar', 'Angola', 'Belize',
                    'Oman', 'Belarus', 'Bhutan', 'Libya', 'Kuwait', 'Trinidad and Tobago',
                    'Djibouti', 'Maldives', 'Cuba', 'Haiti', 'Rwanda', 'South Sudan',
                    'Syria', 'Central African Republic', 'Eswatini', 'Somaliland region',
                    'Guyana', 'Yemen', 'Suriname', 'Burundi', 'Lesotho', 'Sudan', 'Somalia']

# Filter the dataframe so only countries we want are left
df_pre2023 = df_pre2023[~df_pre2023 ['country'].isin(remove_countries)]

### Model formatting

To avoid data leakage, we need to split variables into indepdent and dependent variables then perform one-hot encoding. We also need to split our data into training and test sets before scale our data.

### Split the data into dependent and independent variables




Let's format the data so we can be ready to create a support vector machine.

In this case, the variable we want to predict is data from the 'happiness' coloumn. We will store this in **y**.

The remaining other coloumns will correspond to our classification variables. This data will be store in **X**.

So the independent variables are: `'country', 'year', 'GDP', 'socialSupport',
       'lifeExpectancy', 'freedom', 'generosity', 'corruption'`

In [ ]:
# we will make a copy of our df so as to not modify the original.

X = df_pre2023.drop('happiness', axis=1).copy()
X_2023 = df_2023.drop('happiness', axis=1).copy()

In [ ]:
y = df_pre2023['happiness'].copy()
y_2023 = df_2023['happiness'].copy()

### One-hot encoding **EDIT COMMENTS**

In our datasets, we have one categorical column; country. We can't just plug it into our SVM as is, because SVM requires numerical input.

Luckily we can use a trick to bypass this issue by using one-Hot encoding.

One-hot encoding will create a binary column for each category in the feature. The only downside is that the dimensionality of out data will increase since we have 195 countries in the world. Even though we could maintain dimensionality by using ordinal encoding, we will not use it as it would assign a value to each country, implying a ranking--which is not true in our case. It would, for an example, cluster country 193, 194, 195 together because it would assume they are similiar.

This type of data pre-procesing needs to happen after data splitting to prevent "data leakage".

In [ ]:
X_encoded = pd.get_dummies(X, columns=['country'])
X_encoded_2023 = pd.get_dummies(X_2023, columns=['country'])
X_encoded.head()

You can see that the outcome of hot-encoding is that, for example, the row corresponding to the UAE has a 0 for all country columns (i.e., country_Afghanistan, country_Albania, country_Algeria) except for the column country_United_Arab_Emirates--this is how we maintain the country information for each column in a numerical format.

Since our model performed really well at predicting this year, lets forcast the happiness of contries in the upcoming year.

**check which countries have missing years** Even though the survey claims to have included data about 156 countries, there are only a small number of countries for which data has been recorded in the initial years.



### Splitting into training and test set & scaling **EDIT COMMENTS**

NOTE: Another reason why we already split the dataset into a training and test dataset is to avoid data leakage while scaling. Data leakage in this case means that the information about the training set corrupts the testing one.

K-Fold Cross Validation is a popular resampling technique used to evaluate models on a limited data sample. The primary goal of cross-validation is to prevent overfitting, but it also gives insight into how the model will generalize to an independent dataset.

In K-Fold Cross Validation, the original sample is randomly partitioned into k equal sized subsamples (or "folds"). Of the k subsamples, a single subsample is retained as the validation data for testing the model, and the remaining k-1 subsamples are used as training data. The cross-validation process is then repeated k times, with each of the k subsamples used exactly once as the validation data. The k results can then be averaged to produce a single estimation.

When k equals 10 (i.e., 10-fold cross validation), this means that the original sample is randomly divided into 10 subsamples. For each of 10 iterations, 9 subsamples are used to train the model and the remaining 1 subsample is used to validate the model (i.e., assess its performance). The final model performance is then typically the average performance across the 10 iterations.

This method is particularly useful when you have a small dataset because it maximizes both the test and training data. However, it also has a higher computational cost as the model needs to be trained k times.

Another variation of K-Fold Cross Validation is Stratified K-Fold Cross Validation where each set contains approximately the same percentage of samples of each target class as the complete set, or in case of prediction problems, the mean response value is approximately equal in all the folds. This is generally a better option as it takes imbalance in the data distribution into account.

The `random_state` parameter is a way to set the seed for the pseudo-random number generator used in the algorithm, particularly for methods that involve random processes such as random subsampling, randomly initializing parameters, etc.

The actual number you use as a seed doesn't matter so much. What's important is that the same seed will result in the same random sequence, providing reproducibility for your analysis.

So if you set `random_state` to a high number, it won't affect the quality of your model or its ability to generalize. It will just start the random number generator at a different point in its sequence compared to if you used a lower seed. But as long as you use the same seed each time (whatever that number may be), you'll get the same results each time you run your code.

If you don't set `random_state`, each time you run your code you may get different results, because the seed for the random number generator will be set randomly (often using the current time), causing your code to start at a different point in the random sequence each time it's run.

The most important consideration with `random_state` is to set it the same way each time if you want reproducible results, not whether the seed is a small or large number.

Creating a "fold" in the context of k-fold cross-validation refers to splitting the data into several subsets of roughly equal size.

For instance, when you perform 10-fold cross-validation, you divide your dataset into 10 "folds" or subsets. If you have 1000 samples in your dataset, each fold will have approximately 100 samples.

For each round of cross-validation, one of these folds is used as the testing set and the remaining folds are combined to form the training set. The model is trained on the training set and evaluated on the testing set.

This process is repeated 10 times (or however many folds you have), each time with a different fold being used as the test set. This way, each sample in your dataset is included in the test set exactly once.

The advantage of k-fold cross-validation is that it allows you to use all of your data for both training and testing, which can lead to more robust estimates of the model's performance.

Here's an example with 5-fold cross-validation:

- Round 1: Train on folds 2, 3, 4, 5; test on fold 1
- Round 2: Train on folds 1, 3, 4, 5; test on fold 2
- Round 3: Train on folds 1, 2, 4, 5; test on fold 3
- Round 4: Train on folds 1, 2, 3, 5; test on fold 4
- Round 5: Train on folds 1, 2, 3, 4; test on fold 5

Each "fold" is used once as the test set, and the model's performance is averaged across the 5 rounds to get the final model performance.

In [ ]:
# lets create folds and tell the KFold function whether or not to shuffle the data beforehand.
kf = KFold(n_splits=10, shuffle=True, random_state=1)

The Radial Basis Function (RBF) kernel we are using for our Support Vector Machine assumes that the data is centered and scaled. The RBF kernel measures the similarity between samples based on the Euclidean distance, and if the data is not scaled, features with larger scales can dominate the distance computation and have a disproportionate impact on the model.

The scaling process involves subtracting the mean of each feature and dividing it by the standard deviation. In other words, each column should have a mean value of 0 and a standard deviation of 1. We need to do this in both the training and testing datasets.

Scaling data is a form of preprocessing, and generally, preprocessing steps should be performed within each fold of the cross-validation process. If you perform scaling before the cross-validation, the scaling parameters (mean, standard deviation for standard scaling, for instance) would be computed using all of the data, including the test data that's supposed to be "unseen". This could lead to information leakage from the test set into the training set, which may lead to overly optimistic performance metrics.

The correct way is to compute the scaling parameters (i.e., fit the scaler) on the training set of each fold, then apply those parameters to scale the data in both the training and test sets.

Here's a general procedure using `StandardScaler` and `KFold` in scikit-learn as an example:


In this way, each fold gets its own "view" of the world that is not contaminated by the test data.

This process becomes easier when you use `Pipeline` in scikit-learn, which allows you to define a pipeline of transformations that end with a classifier or regressor. When using `cross_val_score` or other similar functions with a pipeline, the fitting of the preprocessing steps will automatically happen on the training data of each fold.

In [ ]:
# Initialize the scaler
scaler = StandardScaler()

# Prepare empty lists to hold the training and testing data for each fold
X_train_list, X_test_list = [], []
y_train_list, y_test_list = [], []

for train_index, test_index in kf.split(X_encoded):
    # Split the data
    X_train, X_test = X_encoded.iloc[train_index], X_encoded.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Fit the scaler on the training set and transform both sets
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Append to the lists
    X_train_list.append(X_train)
    X_test_list.append(X_test)
    y_train_list.append(y_train)
    y_test_list.append(y_test)

## Initializing support vector machine **EDIT COMMENTS**

Our problem is a regression one, not a classification one since we are working with continuous target values (e.g., predicting a numerical value, in our case, happiness).

In [ ]:
# Initialize the SVR model
model = SVR()

# Create an empty list to store scores for each fold
r2_scores = []
rmse_scores = []
accuracy_scores = []

# Define the threshold
threshold = 0.1  # 10% acceptable limit of error (the percentage of the true value that you'll allow as error)

for i in range(kf.get_n_splits()):
    # Get the data for this fold
    X_train = X_train_list[i]
    X_test = X_test_list[i]
    y_train = y_train_list[i]
    y_test = y_test_list[i]

    # Fit the model and make predictions
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Calculate and store the performance metrics
    r2_scores.append(r2_score(y_test, y_pred))
    rmse_scores.append(np.sqrt(mean_squared_error(y_test, y_pred)))

    # computing accuracy
    relative_error = np.abs((y_test - y_pred) / y_test) # Compute the relative error
    correct_predictions = np.sum(relative_error <= threshold) # Count the number of predictions that fall within the threshold
    accuracy_scores.append(correct_predictions / len(y_test)) # Compute and store the accuracy for this fold

# Print average performance metrics
print("Average R^2 Score:", np.mean(r2_scores))
print("Average RMSE:", np.mean(rmse_scores))
print('Average Accuracy: ', np.mean(accuracy_scores)) # Print average accuracy

So far we defined and fit the SVM regression model within each fold on the pre-2023 data. Currently, out of the box, our SVM model performed pretty well:

 R^2 = `0.889`, RMSE = `0.372`, Accuracy = `85.1%`

Accuracy: If the difference between the predicted and actual values falls within this threshold, then the prediction is deemed correct.

Computing averages for the performance metrics across the multiple folds of a cross-validation process gives us an overall understanding of the model's performance, while reducing the variance and bias that might come from a single split of the data.

Here are some more detailed reasons:

1. **Generalization**: The average performance metrics provide a more general evaluation of the model's ability to predict new, unseen data. Since in k-fold cross-validation each fold serves as a test set once, we are essentially testing the model's performance on the entire dataset, which should give us a good estimate of how well the model will perform on new data.

2. **Stability**: Averaging metrics over multiple folds helps to ensure that the evaluation is stable and reliable, not overly dependent on a particular random split of the data. The variance of the estimate decreases as the number of folds increases.

3. **Bias/Variance Trade-off**: Cross-validation allows us to balance bias and variance. A model that performs well on average across many different splits of the data is less likely to be suffering from high bias (underfitting) or high variance (overfitting), as both of these issues would likely cause poor performance on at least some of the folds.

So, the averaging process in k-fold cross-validation serves to give a more reliable and less biased estimate of the model's true performance than would be obtained from a single train/test split.

## Tune/optimize model parameters

Though our model perfromed quite well, let's find the optimal values for our model paramerts to see if we can improve performance.


So let's use cross validation with the hopes of improving the accuracy on the **testing dataset** from the pre-2023 data.

Since we have more than one parameter to optimize, we will use GridSearchCV() to test all possible combinations.

**Epsilon**: SVR uses the epsilon parameter to specify the epsilon-tube within which no penalty is associated in the training loss function with points predicted within a distance epsilon from the actual value.
**C**: Regularization parameter

The below block of code takes ~10min to run. Hence it is commented out for now, but below are the ideal parameters found after running it last time.

We see that the ideal parameter values are C = `100`, epsilon = `0.1`, gamma= `'auto'`, and kernel = `rbf`.


In [ ]:
# # Define the grid of parameters to search
# param_grid = [
#     {
#         'C': [0.5, 1, 10, 100], # Note: Values of C > 0
#         'gamma':['scale', 'auto', 1, 0.1, 0.001, 0.0001],
#         'epsilon': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
#         'kernel': ['rbf'] # 'linear' was also previously added, but since gridsearch computed rbf as optimal, we will remove 'linear' to reduce future run time.
#     },
# ]
# # We are including C=1 and gamma=0.1 since they are the default values

# # Initialize the grid search
# grid_search = GridSearchCV(model,
#                            param_grid,
#                            cv=kf, # cross-validation splitting strategy. the function will use the k-fold cross-validation strategy defined by the kf object.
#                            scoring='neg_mean_squared_error',
#                            n_jobs=-1) # n_jobs, the higher number, more messages

# # Conduct the grid search
# grid_search.fit(X_encoded, y)

# print(grid_search.best_params_) # print the optimal parameters

## Building final support vector machine

In [ ]:
# Initialize the SVR model
model = SVR(kernel='rbf', C = 100, epsilon = 0.1, gamma = 'auto')

# Create an empty list to store scores for each fold
r2_scores = []
rmse_scores = []
accuracy_scores = []

# Define the threshold
threshold = 0.1  # 10% acceptable limit of error (the percentage of the true value that you'll allow as error)

for i in range(kf.get_n_splits()):
    # Get the data for this fold
    X_train = X_train_list[i]
    X_test = X_test_list[i]
    y_train = y_train_list[i]
    y_test = y_test_list[i]

    # Fit the model and make predictions
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Calculate and store the performance metrics
    r2_scores.append(r2_score(y_test, y_pred))
    rmse_scores.append(np.sqrt(mean_squared_error(y_test, y_pred)))

    # computing accuracy
    relative_error = np.abs((y_test - y_pred) / y_test) # Compute the relative error
    correct_predictions = np.sum(relative_error <= threshold) # Count the number of predictions that fall within the threshold
    accuracy_scores.append(correct_predictions / len(y_test)) # Compute and store the accuracy for this fold

# Print average performance metrics
print("Average R^2 Score:", np.mean(r2_scores))
print("Average RMSE:", np.mean(rmse_scores))
print('Average Accuracy: ', np.mean(accuracy_scores)) # Print average accuracy

Without tuning:  R^2 = `0.889`, RMSE = `0.372`, Accuracy = `85.1%`

Aftering tuning: R^2 = `0.892`, RMSE = `0.364`, Accuracy =  `85.1%`

The SVM preformed really well straight out of the box, with only slight improvements being added to the R^2 and RSME after optimization, but no changce in accuracy.


## Evaluation: How close is our model predictions to actual 2023 data?

In [ ]:
# Predict the 2023 data using the trained model
y_pred_2023 = model.predict(scaler.transform(X_encoded_2023))

# Create a scatter plot
plt.figure(figsize=(5, 5))
plt.scatter(y_2023, y_pred_2023, alpha=0.5)
plt.xlabel('Actual happiness scores from 2023')
plt.ylabel('Predicted happiness scores by SVM model')
plt.title('Predicted 2023 Happiness Scores as a Function of Actual Scores')

# Line of equality
plt.plot([min(y_2023), max(y_2023)], [min(y_2023), max(y_2023)], color='red')

# annotate
avg_r2 = np.mean(r2_scores)
avg_rmse = np.mean(rmse_scores)
plt.annotate(f'R^2 = {avg_r2:.2f}\nRMSE = {avg_rmse:.2f}', xy=(0.75, 0.05), xycoords='axes fraction')

plt.show()

1. The points fall along the line of equality. The closer the points are to this line, the more accurate the model's predictions are.
2. There is a tight cluster of points around the line of equality, implying a more precise model.
3. There is no systematic pattern to the points deviating from the line of equality (for example, if points for lower actual values are consistently overpredicted), this could indicate that the model does not have a bias / captures the relationship between the predictors and the outcome variable.

## Further performance evaluations

### Does our prediction model have a lot of error?


 visualizes the errors in out predictions. Ideally, residuals should be randomly and evenly distributed around the x-axis, and shouldn't form any discernible pattern.

In [ ]:
residuals = y_test - y_pred
plt.scatter(y_pred, residuals)
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.title('Residual Plot')
plt.show()

### How much are our predictions off by?
**What does the distribution mean in plain words?**

In [ ]:
sns.histplot(residuals, kde=True)
plt.xlabel('Residuals')
plt.title('Distribution of Residuals')
plt.show()

* Hard to predict the future! Just because our model performed well on 2023 data, there is no guarantee it will perform as well on the next year's data. The underlying relationships between variables might change over time. A model can make predictions for 2024, but we won't be able to validate these predictions against actual values as we did for the previous years, because the real 2024 data is not available yet.

* If there are large year-to-year fluctuations in happiness scores or the predictors, the model might not produce accurate predictions for 2024.
* the predictions are essentially a replication of 2023's data with some minor alterations due to the model. They don't account for potential new information or significant changes that may occur in 2024.

# **Q2:** Can we predict 2024 happiness?

Since our model performed so well, it is reasonable to try an use to forecast the upcoming year.

## Format data

To avoid data leakage, we need to split our data into training and test sets before scale our data.

### Split the data into dependent and independent variables

Where as in the 2023 model, the training and test dataset did not include 2023 data. In this model, 2023 data is included too.

In [ ]:
# we will make a copy of our df so as to not modify the original.
X = df.drop('happiness', axis=1).copy()
y = df['happiness'].copy()

### One-hot encoding

In [ ]:
X_encoded = pd.get_dummies(X, columns=['country'])
X_encoded.head()

### Splitting into training and test set & scaling

In [ ]:
# lets create folds and tell the KFold function whether or not to shuffle the data beforehand.
kf = KFold(n_splits=10, shuffle=True, random_state=1)

# Initialize the scaler
scaler = StandardScaler()

# Prepare empty lists to hold the training and testing data for each fold
X_train_list, X_test_list = [], []
y_train_list, y_test_list = [], []

for train_index, test_index in kf.split(X_encoded):
    # Split the data
    X_train, X_test = X_encoded.iloc[train_index], X_encoded.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Fit the scaler on the training set and transform both sets
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Append to the lists
    X_train_list.append(X_train)
    X_test_list.append(X_test)
    y_train_list.append(y_train)
    y_test_list.append(y_test)

## Building support vector machine

In [ ]:
# Initialize the SVR model
model = SVR(kernel='rbf', C = 100, epsilon = 0.1, gamma = 'auto')

# Create an empty list to store scores for each fold
r2_scores = []
rmse_scores = []
accuracy_scores = []

# Define the threshold
threshold = 0.1  # 10% acceptable limit of error (the percentage of the true value that you'll allow as error)

for i in range(kf.get_n_splits()):
    # Get the data for this fold
    X_train = X_train_list[i]
    X_test = X_test_list[i]
    y_train = y_train_list[i]
    y_test = y_test_list[i]

    # Fit the model and make predictions
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Calculate and store the performance metrics
    r2_scores.append(r2_score(y_test, y_pred))
    rmse_scores.append(np.sqrt(mean_squared_error(y_test, y_pred)))

    # computing accuracy
    relative_error = np.abs((y_test - y_pred) / y_test) # Compute the relative error
    correct_predictions = np.sum(relative_error <= threshold) # Count the number of predictions that fall within the threshold
    accuracy_scores.append(correct_predictions / len(y_test)) # Compute and store the accuracy for this fold

# Print average performance metrics
print("Average R^2 Score:", np.mean(r2_scores))
print("Average RMSE:", np.mean(rmse_scores))
print('Average Accuracy: ', np.mean(accuracy_scores)) # Print average accuracy

 R^2 = `0.898`, RMSE = `0.358`, Accuracy =  `86.3%`

The SVM preformed really well.


## Preparing X variables for 2024 forecasting

After training our Support Vector Machine model on the data from 2005-2023, we will use this trained model to forecast the 'happiness' score (y) for the year 2024.

However, in order to make this prediction, we will first need the predictor variables (X) for the year 2024.

Since the predictor variables for 2024 are time-independent, we can simply extract the most recent available data for those variables.

In [ ]:
X_2024 = df[df['year'] == 2023] # Get the most recent data for each country
X_2024 = X_2024.drop('happiness', axis=1) # Remove the 'happiness' column, as it's the target variable

# we need to make sure X_2024 is processed in the same way as our training data (2005-2023)

# One-hot encode the 'country' column to match the training data
X_2024_encoded = pd.get_dummies(X_2024, columns=['country'])

# Ensure that the 2024 data has the same columns as the training data
missing_cols = set(X_encoded.columns) - set(X_2024_encoded.columns)
for c in missing_cols:
    X_2024_encoded[c] = 0

# Make sure the columns are in the same order as in the training data
X_2024_encoded = X_2024_encoded[X_encoded.columns]

# Scale the 2024 data using the same scaler object used on the training data
X_2024_scaled = scaler.transform(X_2024_encoded)

In [ ]:
# use the fitted model to predict 2024's happiness scores
y_pred_2024 = model.predict(X_2024_scaled)

# Create a bar plot for the predicted 2024 happiness scores
plt.figure(figsize=(25, 5))
plt.scatter(X_2024['country'], y_pred_2024)
plt.xlabel('Country')
plt.ylabel('Predicted happiness score in 2024')
plt.title('Predicted 2024 Happiness Scores for Each Country')
plt.xticks(rotation=90)  # Rotate x-axis labels for better readability
plt.show()

In [ ]:
# Create a new DataFrame with the country names, the 2023 values, and the predicted 2024 values
df_2024 = pd.DataFrame({
    'country': X_2024['country'],
    '2023_value': df[df['year'] == 2023]['happiness'],  # Get 2023 happiness values from the original dataframe
    'predicted_2024_value': y_pred_2024
})

# Calculate the percent change from 2023 to 2024
df_2024['percent_change'] = ((df_2024['predicted_2024_value'] - df_2024['2023_value']) / df_2024['2023_value']) * 100

In [ ]:
plt.figure(figsize=(10, 30))
sns.barplot(data=df_2024.sort_values('percent_change', ascending=False), y='country', x='percent_change')
plt.xlabel('Percent Change')
plt.ylabel('Country')
plt.title('Percent Change in Happiness from 2023 to Predicted 2024')
plt.show()

The plot is quite large, and if we scaled it down the text will no longer be legible. Instead, we will create two subplots to represent percentage change from 2023 to predicted 2024.

In [ ]:
# Split the data into positive and negative percent changes
df_2024_positive = df_2024[df_2024['percent_change'] > 0].sort_values('percent_change', ascending=False)
df_2024_negative = df_2024[df_2024['percent_change'] < 0].sort_values('percent_change')

# Create a subplot with two plots
fig, axs = plt.subplots(nrows=2, figsize=(10, 20))

# Create the bar plot for positive percent changes
sns.barplot(data=df_2024_positive, y='country', x='percent_change', ax=axs[0], color='lightgreen')
axs[0].set_xlabel('Change in happiness (%)')
axs[0].set_ylabel('Country')
axs[0].set_title('Positive Percent Change in Happiness from 2023 to Predicted 2024')

# Create the bar plot for negative percent changes
sns.barplot(data=df_2024_negative, y='country', x='percent_change', ax=axs[1], color='salmon')
axs[1].set_xlabel('Change in happiness (%)')
axs[1].set_ylabel('Country')
axs[1].set_title('Negative Percent Change in Happiness from 2023 to Predicted 2024')

# Display the plots
plt.tight_layout()
plt.show()

## Limitations

#  **Q3:** Which variables are most strongly associated with happiness?



Looking at the forecast for global happiness in the upcoming year is interesting. For an example, the UAE is expected to witness the largest positive change in their happiness compared to other countries.

What would be even more interesting, is to investigate what are the most salient features that associated with predicting happiness on a global scale. What about the UAE, are the variables that best predict happiness for the UAE population different than remaining countries?

Support Vector Machines are not typically used for feature selection or understanding feature importance because they don't provide direct access to which features are most important. This is because SVMs are more of a "black box" model where interpretability is difficult.



*   Are there any surprising variables that have a significant effect on happiness?

Knowing what variables contribute to citizens' happiness can help governments develop and prioritize policies that enhance those factors. For instance, if variables related to health and education have strong correlations with happiness, then investing more in public health and education policies could lead to a happier population.

Understanding happiness can be important for healthcare providers too. Unhappiness can be both a symptom and a cause of poor health, so healthcare policies might benefit from considering happiness levels in the population.

Traditional economic measures like GDP have long been used as a proxy for societal success, but they don't capture everything that matters to people's lives. A happiness model could provide an additional measure to guide economic decision-making.




## Which variables have the largest influence on determining happiness? **EDIT comments**

We can fit a simple linear regression model to our data and create a plot of residuals (the difference between the observed and predicted values) versus the predicted values. If we see patterns in this plot, it suggests that the relationship between the predictors and the target variable is not entirely linear.

Permutation Importance is a great way to measure the importance of different features in your model. You can use this approach to help interpret your model and understand which features are driving the predictions.

The idea behind Permutation Importance is simple: for each feature, we randomly shuffle the values of that feature and measure how much the model's performance drops. If shuffling a feature's values leads to a big drop in the model's performance, that feature is important; if there's little change, the feature is not important.

You would typically use Permutation Importance on your validation or test data, because it can help you understand which features your model is actually using to make predictions on unseen data. It would be less informative to use Permutation Importance on the training data, because it wouldn't give you insight into how the model performs on new data.

**Permutation Importance**
This technique can help you determine the most influential features in your SVM model by permuting the values of each feature and scoring the impact on model performance. This could be done using sklearn's permutation_importance().

Permutation importance is a useful tool for understanding the impact of a specific feature on the predictions of a machine learning model like SVM. It works by permuting the values of each feature one by one and measuring the decrease in the model's performance. The features that cause the biggest drop in performance are considered the most important.

However, there are a few things to keep in mind when using permutation importance:

Correlated Features: If two features are highly correlated, permuting one of them might not lead to a big drop in performance because the model can still use the other feature to make predictions. In this case, the importance of both features might be underestimated.
Interactions between Features: Permutation importance measures the importance of each feature in isolation and does not take into account interactions between features. If the effect of one feature depends on the value of another feature, this interaction effect won't be captured by permutation importance.
Non-linear and complex relationships: SVMs and other complex models can capture non-linear relationships and interactions between features, which may not be reflected entirely by permutation importance.


[Reference](https://medium.com/@T_Jen/feature-importance-for-any-model-using-permutation-7997b7287aa) used to write this code.

In this case, taking an average of the training sets doesn't make much sense due to the nature of cross-validation. Each set of training data is different because of the shuffling of data, hence it's not advisable to average them.

Instead of averaging the training sets, you should compute the permutation importance on each fold of your cross-validation and then average those importances.

Please note that permutation importance should ideally be calculated on the validation set or unseen data to provide an unbiased estimate of the model performance.

In this code, we calculate the permutation importances for each split in the KFold cross-validation. Then we store these importances in a DataFrame, which allows us to easily calculate the average importance for each feature. Finally, we sort the DataFrame by the average importance to easily see which features are most important. Please note, features with small or zero weights might not be important.

Data leakage can occur when information that would not be available at prediction time is used when building the model. This can lead to overly optimistic performance estimates.

In the provided code, there doesn't seem to be obvious data leakage. You are fitting the model on the training set and then computing permutation importances on the test set, which is the right way to do it.

However, the concept of data leakage with Permutation Importance is a nuanced one. If you calculate Permutation Importance using the test set (as you are doing), it can provide insight into what features were most important for generalizing from the training set to unseen data. But, it could be argued that this is a form of data leakage because you're using the test set to influence your interpretation of the model.

From a strict perspective, the test set should only be used for the final evaluation, and any form of analysis or interpretation of the model (including Permutation Importance) should be done on a separate validation set or using cross-validation on the training set. This prevents any knowledge of the test set from influencing the model or interpretation.

Therefore, while your current approach isn't incorrect, it could be made more robust by calculating Permutation Importance on a separate validation set or using a cross-validation strategy.

 To compute permutation importance on validation sets during cross-validation, we can use the same logic and integrate it into our cross-validation loop.

 This way, we are not using our test set for the calculation of permutation importance but rather using the validation sets from the cross-validation. Note that the `X_test` and `y_test` are essentially acting as validation sets here, as they are different subsets of our training data in each iteration of the cross-validation.



In [ ]:
# expect run time to take long (~5-8 minutes)

# Initialize a dataframe to store importances for each fold
importances_df = pd.DataFrame()

# Loop over each split
for i in range(kf.get_n_splits()):

    # Get the data for this fold
    X_train = X_train_list[i]
    X_test = X_test_list[i] # In this case, X_test acts as a validation set
    y_train = y_train_list[i]
    y_test = y_test_list[i] # And y_test acts as validation labels

    # Fit the model
    model.fit(X_train, y_train)

    # Compute the permutation importances
    perm = PermutationImportance(model, random_state=1).fit(X_test, y_test)

    # Get feature names
    feature_names = X_encoded.columns.tolist()

    # Get importances and convert them into a dataframe
    importances = pd.DataFrame(perm.feature_importances_, index=feature_names, columns=[f'Importance Fold {i}'])

    # Concatenate the current fold's importances with the previous ones
    importances_df = pd.concat([importances_df, importances], axis=1)

# Compute the average importances
importances_df['Average Importance'] = importances_df.mean(axis=1)

# Sort the importances
importances_df = importances_df.sort_values('Average Importance', ascending=False)

# Print the importances
print(importances_df)

Including the 'country' as a feature in our model can provide it with additional information about cultural, social, economic, and political differences between countries, which are not captured by the other predictors in our dataset.

However, it's not always straightforward to interpret the effect of the 'country' feature in our model. For instance, 'country_Singapore' having a positive importance means that, all other things being equal, being in Singapore is associated with a higher predicted happiness score compared to the reference country (which, in one-hot encoding, is typically implicitly defined as the country left out of the encoding, or a sort of "average" country effect).

Also, it's important to note that 'country_Singapore' having a non-zero importance doesn't mean that Singapore is more important to global happiness than other countries. Rather, it means that Singapore's happiness score is more different from the reference country's score than other countries are. Other countries may have zero or near-zero importances because their happiness scores are very similar to the reference country's score.

The importance of a feature reflects how much the model's predictions change when you shuffle the values of that feature, thereby destroying the relationship between the feature and the target variable. If the model's predictions change a lot, then the feature is important; if they don't change much, the feature is not important. This method of computing feature importance is robust and can capture complex, nonlinear relationships between features and the target variable.

As such, including the 'country' as a feature might be beneficial for our model if there are country-specific factors that affect happiness and are not captured by our other features. On the other hand, if our other features already capture all of the relevant country-specific factors, then including 'country' as a feature might not improve our model.

In [ ]:
importances_df.head()

In [ ]:
print(importances_df.columns)

In [ ]:
importances_df = importances_df[['Average Importance']]
importances_df.head()

We want to create a visualization, but we want the countries to have different color than the other variables i.e., GDP, for clarity.

In [ ]:
# The pandas function str.contains can be used to create a boolean mask indicating
# whether each index contains the string 'country'. This mask can then be used to filter the DataFrame.

# Create a boolean mask
mask = importances_df.index.str.contains('country')

# Create DataFrame for features containing 'country'
df_importanceCountry = importances_df[mask]

# Create DataFrame for features not containing 'country'
df_importanceNon_country = importances_df[~mask]

In [ ]:
# Remove 'country_' from the index
df_importanceCountry.index = df_importanceCountry.index.str.replace('country_', '', regex=False)
df_importanceCountry.head()

In [ ]:
# lets create a color map
def color_negative_red(value):
  """
  Colors values in a DataFrame green if they are positive and red if they are negative.

  Args:
    value (float): The value to color.

  Returns:
    str: The color string.
  """
  if value < 0:
    color = 'red'
  else:
    color = 'green'

  return 'color: %s' % color

In [ ]:
# Apply the color map to dataframe for feature imoportance by country
styledCountry_df = df_importanceCountry[['Average Importance']].style.applymap(color_negative_red)

styledCountry_df

In [ ]:
# Apply the color map to dataframe for feature imoportance by scores
styledScores_df = df_importanceNon_country[['Average Importance']].style.applymap(color_negative_red)

styledScores_df

## Are there any interacting variables that predict happiness?

Pairplots can be used to visualize potential interactions between features in relation to the target variable. To create this we just need  a 3D plot that can illustrate the relationship between two features and the target variable (happiness).

In [ ]:
# the feature names are from this df
print(X_encoded.columns)

In [ ]:
# what are all the possible combinations of features that we can plot?
# the features we are interested in are: GDP, socialSupport, corruption, freedom, generosity, year,and, lifeExpectancy

from itertools import combinations

features = ['GDP', 'socialSupport', 'corruption', 'freedom', 'generosity', 'year', 'lifeExpectancy']

# Get all possible combinations of 2 features
feature_combinations = list(combinations(features, 2))

# let's print them
for combo in feature_combinations:
    print(combo)

In [ ]:
# above we found all the possible combinations. So how many combination in total can we plot?
print(len(feature_combinations))

We have 21 possible plots to make. We could manually make a plot for each pair, but lets make our life easier and just use a loop to do it all at once.

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from itertools import combinations

selected_features = ['GDP', 'socialSupport', 'corruption', 'freedom', 'generosity', 'year', 'lifeExpectancy']

# Get all possible combinations of 2 features from the selected features
selected_feature_combinations = list(combinations(selected_features, 2))

# we already fit the model and made predictions earlier for research question #2 called 'y_pred'
y_pred_full = model.predict(X_encoded)

for pair in selected_feature_combinations:
    feature1, feature2 = pair

    # Create a new figure
    fig = plt.figure(figsize=(3, 3))

    # Add a 3D subplot
    ax = fig.add_subplot(111, projection='3d')

    # Create a scatter plot with 'feature1' and 'feature2' as the x and y axes, and the predicted target as the z axis
    ax.scatter(X_encoded[feature1], X_encoded[feature2], y_pred_full)

    # Set labels for the axes
    ax.set_xlabel(feature1)
    ax.set_ylabel(feature2)
    ax.set_zlabel('Predicted Happiness')

    # Set title for the plot
    ax.set_title(f'3D scatter plot for features {feature1} and {feature2}')

    plt.show()

These plots are static, so it is hard to visually inspect. Let's make an interactive plot instead.

In [ ]:
import plotly.graph_objects as go
from itertools import combinations
import plotly.io as pio

pio.renderers.default='colab' # enables 3D interactive plots in Google Colab using plotly

# Loop through the pairs of features
for pair in selected_feature_combinations:
    feature1, feature2 = pair

    trace = go.Scatter3d(
        x = X_encoded[feature1],
        y = X_encoded[feature2],
        z = y_pred_full,
        mode = 'markers',
        marker = dict(
            size = 4,
            color = y_pred_full,
            colorscale = 'Viridis'
        )
    )

    data = [trace]
    layout = go.Layout(
        title = f'3D scatter plot for features {feature1} and {feature2}',
        scene = dict(
            xaxis = dict(title = feature1),
            yaxis = dict(title = feature2),
            zaxis = dict(title = 'Predicted Happiness')
        )
    )

    fig = go.Figure(data=data, layout=layout)
    fig.show()


# Focusing on the Governance-Quality Measures based on Data from the Worldwide Governance Indicators (WGI) Project

# Pending

## Scatter diagram of the distribution between each parameter

## Correlation matrix between all the features

why? Correlation is used to identify the pairwise correlation of all columns that exist in the dataset. By using heatmap, it explicitly exhibits the attributes which are strongly correlated to each other.

## Which country is performing the best?

Happiness can be a useful metric for international comparisons. Countries often strive to learn from each other's successes and failures, and understanding the factors that contribute to happiness can provide valuable insights.

### Overall happiness

### High performers in each independent variable

### Heatmap of Global happiness

Geographical Visualization of Happiness Score: to help to comprehend trends in our data, identify outliers as well as other areas of interest

# Limitations

While our model could be very valuable, it's important to remember that these are complex issues, and that the model's predictions should ideally be one part of a larger decision-making process. It's also crucial to consider factors such as data quality and representation, the potential for overfitting, and ethical considerations.

**What are the reason a machine learning model can perform really well, but the dimension in the data doesnt capture complex behaviour.**

# Conclusions

As a consequence, SVM regression might be used to determine the happiness of a country. If happiness can be predicted at an early stage, it is beneficial to stakeholds to prevent a decline.

Maximize life satisfaction.